Rename a bunch of files using Git and renamer?



In [3]:
var execSync = require('child_process').execSync;
try {
    require.resolve('renamer');
} catch (e) {
    execSync('npm install renamer');
}
var renamer = require('renamer');
var renameUsingGit = (root, match, find, replace) => {
    var files = renamer.expand(path.join(root, match));
    var results = renamer.replace({
        files: files.filesAndDirs, 
        find: find, 
        replace: replace});
    return renamer.dryRun(results).list.map(r => {
        // rename with git instead
        var output = execSync('git ' 
                 + '--work-tree=' + JSON.stringify(root)
                 + ' --git-dir=' + JSON.stringify(path.join(root, '.git'))
                 + ' mv ' + JSON.stringify(r.before) 
                 + ' ' + JSON.stringify(r.after));
        return output.toString() || r.after;
    });
};
(renameUsingGit);


In [4]:
// test it
// rename all scss files to less
renameUsingGit(
    '/Users/briancullinan/Documents/portal',
    'src/**/*.scss',
    '.scss',
    '.less');

How to use memory-fs and rewire to audit cli events?



In [2]:
var path = require('path');
var execSync = require('child_process').execSync;
try {
    require.resolve('mock-require');
    require.resolve('typescript');
    require.resolve('memory-fs');
} catch (e) {
    execSync('npm install mock-require typescript https://github.com/webpack/memory-fs');
}
// TODO: use https://github.com/jhnns/rewire for replacing variables, something like https://github.com/mariocasciaro/object-path?
var mock = require('mock-require');

var mockTypescriptFs = (data) => {
    var fs = require('fs');
    var fsMock = {};
    var MemoryFileSystem = require('memory-fs');
    var memfs = new MemoryFileSystem(data);
    var funcs = Object.keys(fs)
        .filter(k => typeof memfs[k] == 'function');
    
    var fsMock = Object.assign({}, fs);
    // write operations
    ['writeFile', 'mkdir', 'rmdir', 'unlink', 'createWriteStream',
     'writeFileSync', 'mkdirSync', 'rmdirSync', 'unlinkSync']
        .forEach((k) => {
            fsMock[k] = function () {
                //console.log(k + ' (' + arguments[0] + ') ');
                // create the containing directory first
                if(!memfs.existsSync(path.dirname(arguments[0]))
                    && fs.existsSync(path.dirname(arguments[0]))) {
                    memfs.mkdirpSync.apply(memfs, [path.dirname(arguments[0])]);
                }
                return memfs[k].apply(memfs, arguments);
            }
        });
    
    // read operations
    ['readFile', 'stat', 'exists', 'readlink', 'createReadStream',
     'readFileSync', 'statSync', 'existsSync', 'readlinkSync']
        .forEach((k) => {
            fsMock[k] = function () {
                //console.log(k + ' (' + arguments[0] + ') ');
                // read from underlying filesystem instead
                if(!memfs.existsSync(arguments[0])) {
                    return fs[k].apply(null, arguments);
                }
                return memfs[k].apply(memfs, arguments);
            }
        });
    // these functions fallback to underlying filesystem 
    // if a file does not exist
    
    mock('fs', fsMock);
    
    var ts = require('typescript');
    var tsMock = Object.assign({}, ts);
    var toMock = {
        readFile: 'readFileSync',
        realpath: 'realpathSync',
        writeFile: 'writeFileSync',
        fileExists: 'existsSync',
        directoryExists: 'existsSync',
        createDirectory: 'mkdirSync',
        getDirectories: 'readdirSync'
    };
    Object.keys(toMock)
        .forEach(k => {
            if(typeof fsMock[toMock[k]] === 'function') {
                tsMock.sys[k] = function () {
                    // typescript.sys functions return defined on error
                    try {
                        return fsMock[toMock[k]].apply(memfs, arguments);
                    } catch (e) {
                        return;
                    }
                };
            }
        })
    
    mock('typescript', tsMock);
    
    var isBinaryFile = mock.reRequire('isbinaryfile').sync;
    mock('isbinaryfile', {
        sync: function () {
            if(typeof arguments[0] == 'string') {
                if(!memfs.existsSync(arguments[0])) {
                    var file = fs.readFileSync(arguments[0]);
                    return isBinaryFile(file, file.length);
                }
                return isBinaryFile(memfs.readFileSync(arguments[0]),
                                    memfs.readFileSync(arguments[0]).length);
            }
            return isBinaryFile.apply(null, arguments);
        }
    });
    
    mock.data = () => memfs.data;
    return mock;
};
(mockTypescriptFs);

// TODO: search github for projects like graceful-fs and then search google for projects that use graceful-fs and see if we can record some filesystem activity



In [3]:

mockTypescriptFs();
